In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# 1. 데이터 로드
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# 라벨 컬럼 이름 (실제 대회 라벨 컬럼 이름으로 변경해야 합니다.)
label = 'Cancer'

# 예측 대상 컬럼 이름 (테스트 데이터에 없는 라벨 컬럼 제외)
predict_var = [col for col in test_data.columns if col != label]

# 2. AutoGluon 모델 학습 (시간 제한 설정)
predictor = TabularPredictor(label=label, eval_metric='log_loss').fit(
    train_data, time_limit=3600  # 예시: 최대 학습 시간 1시간
)

# 3. 테스트 데이터 예측 (확률값 예측 - 데이콘 규정에 따라 라벨 예측으로 변경 가능)
# 데이콘에서 확률값 제출을 요구하는지, 아니면 클래스 라벨 제출을 요구하는지 확인해야 합니다.
# 확률값 제출 예시 (클래스가 2개인 경우):
predictions_proba = predictor.predict_proba(test_data[predict_var])

# 예측 라벨 제출 예시:
# predictions = predictor.predict(test_data[predict_var])

# 4. 제출 파일 생성 (데이콘 규정에 맞는 형식으로 수정해야 합니다.)
submission = pd.DataFrame()

# 테스트 데이터에 ID 컬럼이 있다면 사용, 없다면 임의의 ID 생성 필요
if 'id' in test_data.columns:
    submission['id'] = test_data['id']
else:
    submission['id'] = range(len(test_data)) # 예시: ID 컬럼이 없는 경우

# 예측 확률값 컬럼 이름 (실제 대회 규정에 맞게 변경해야 합니다.)
# 클래스가 2개인 경우, 양성 클래스의 확률을 제출하는 경우가 많습니다.
if 'predictions_proba' in locals():
    # AutoGluon은 클래스 이름으로 컬럼을 생성할 수 있습니다.
    # 예를 들어, 클래스 이름이 0과 1이라면 predictions_proba는 두 개의 컬럼을 가질 수 있습니다.
    # 데이콘에서 요구하는 양성 클래스 컬럼 이름을 확인하여 사용해야 합니다.
    positive_class_label = predictor.class_names[1] if len(predictor.class_names) > 1 else predictor.class_names[0]
    submission[label] = predictions_proba[positive_class_label]
else:
    # 예측 라벨 제출 시 (데이콘 규정에 따라 데이터 타입 변환 필요할 수 있음)
    submission[label] = predictions.astype(int) # 예시: 예측 라벨을 정수형으로 변환

# 5. 제출 파일 저장 (submission4.csv)
submission.to_csv('submission4.csv', index=False)

print("📁 submission4.csv 저장 완료")
print("이제 데이콘에 submission4.csv 파일을 제출해 보세요.")